# Dependency Handling

In [1]:
# Install pytorch

!pip3 install torch torchvision torchaudio

In [2]:
# Install Transformers (using for NLP model - allows for easily import model)
# Install Requests (allow requests to webpages for analysis)
# Install BeautifulSoup (extract relevant data from webpage)
# Install Pandas and Numpy (structure data)

!pip3 install requests transformers beautifulsoup4 pandas numpy

In [3]:
# Tokenizer - Parse through string to get numbers.
# AutoModelForSequenceClassification - Give us architecture from transformers to load in NLP model
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re
import numpy as np
import pandas as pd

C:\Users\dawoo\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Setup Model (Initialization)

In [4]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

# Encoding and Sentiment Calculation

In [5]:
tokens = tokenizer.encode('I loved taking 482X with Dr. Duwe!', return_tensors='pt')

In [6]:
# Debug

# tokens
# tokens[0]
# tokenizer.decode(tokens[0])

In [7]:
result = model(tokens)

In [8]:
result

SequenceClassifierOutput(loss=None, logits=tensor([[-2.1400, -2.0940, -0.7388,  1.3721,  2.8214]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [9]:
star_rating = int(torch.argmax(result.logits)) + 1
print('Star Rating: ', star_rating)

Star Rating:  5


#  Parse Website

In [17]:
req = requests.get('https://www.yelp.com/biz/social-brew-cafe-pyrmont')
soup = BeautifulSoup(req.text, 'html.parser')
regex = re.compile('.*comment.*')
result_website = soup.find_all('p', {'class':regex})
strings = [i.text for i in result_website]
strings

["Great coffee and vibe. That's all \xa0you need. Crab was outstanding but not good finger food like a taco should be. Really want to try the pork belly sandwich - looked excellent. This became my go to breakfast place in Darling harbor. Had the avocado salmon salad breakfast and it was excellent. Service has been excellent.",
 "Great coffee and vibe. That's all \xa0you need. Crab was outstanding but not good finger food like a taco should be. Really want to try the pork belly sandwich - looked excellent.",
 'I came to Social brew cafe for brunch while exploring the city and on my way to the aquarium. I sat outside. The service was great and the food was good too!I ordered smoked salmon, truffle fries, black coffee and beer.',
 "Ricotta hot cakes! These were so yummy. I ate them pretty fast and didn't share with anyone because they were that good ;). I ordered a green smoothie to balance it all out. Smoothie was a nice way to end my brekkie at this restaurant. Others with me ordered th

# Load Website Data into Dataframe and Star Rating

In [11]:
df = pd.DataFrame(np.array(strings), columns=['string'])

# Debug
df['string'].iloc[0]

"Great coffee and vibe. That's all \xa0you need. Crab was outstanding but not good finger food like a taco should be. Really want to try the pork belly sandwich - looked excellent. This became my go to breakfast place in Darling harbor. Had the avocado salmon salad breakfast and it was excellent. Service has been excellent."

In [12]:
def StringToScore(string):
    tokens = tokenizer.encode(string, return_tensors='pt')
    output = model(tokens)
    return int(torch.argmax(output.logits)) + 1
    

In [13]:
# Debug
# StringToScore(df['string'].iloc[0])

In [14]:
df['sentiment'] = df['string'].apply(lambda x: StringToScore(x[:1024]))

In [15]:
df

,string,sentiment
0,Great coffee and vibe. That's all you need. C...,5
1,Great coffee and vibe. That's all you need. C...,4
2,I came to Social brew cafe for brunch while ex...,5
3,Ricotta hot cakes! These were so yummy. I ate ...,5
4,I went here a little while ago- a beautiful mo...,1
5,We came for brunch twice in our week-long visi...,5
6,Ron & Jo are on the go down under and Wow! We...,5
7,Good coffee and toasts. Straight up and down -...,5
8,This place is a gem. The ambiance is to die fo...,4
9,Delicious. The waitress was hot. The burger wa...,4


In [16]:
print("Average Sentiment: ", df['sentiment'].mean())

Average Sentiment:  4.363636363636363
